# Pokemon Analysis (Similarity Matrix + Network Graphing)
Keon Feizy-Marandy & Lexin Deang

In [24]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from pyvis.network import Network
import numpy as np

In [25]:
df = pd.read_csv('pokedex.csv')
df.head()[:5]

,id,name,height,weight,hp,attack,defense,s_attack,s_defense,speed,type,evo_set,info
0,1,bulbasaur,7,69,45,49,49,65,65,45,"{grass,poison}",1,A strange seed was planted on its back at birt...
1,2,ivysaur,10,130,60,62,63,80,80,60,"{grass,poison}",1,"When the bulb on its back grows large, it appe..."
2,3,venusaur,20,1000,80,82,83,100,100,80,"{grass,poison}",1,The plant blooms when it is absorbing solar en...
3,4,charmander,6,85,39,52,43,60,50,65,{fire},2,"Obviously prefers hot places. When it rains, s..."
4,5,charmeleon,11,190,58,64,58,80,65,80,{fire},2,"When it swings its burning tail, it elevates t..."


In [26]:
df.describe()

,id,height,weight,hp,attack,defense,s_attack,s_defense,speed,evo_set
count,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000
mean,513.000000,12.116098,669.865366,70.184390,77.521951,72.507317,70.080976,70.205854,67.186341,253.195122
std,296.036315,12.481673,1212.731138,26.631054,29.782541,29.286972,29.658378,26.639329,28.717227,159.505305
min,1.000000,1.000000,1.000000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.000000
25%,257.000000,5.000000,85.000000,50.000000,55.000000,50.000000,47.000000,50.000000,45.000000,110.000000
50%,513.000000,10.000000,280.000000,68.000000,75.000000,70.000000,65.000000,67.000000,65.000000,257.000000
75%,769.000000,15.000000,700.000000,85.000000,100.000000,90.000000,90.000000,86.000000,88.000000,387.000000
max,1025.000000,200.000000,9999.000000,255.000000,181.000000,230.000000,173.000000,230.000000,200.000000,549.000000


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1025 non-null   int64 
 1   name       1025 non-null   object
 2   height     1025 non-null   int64 
 3   weight     1025 non-null   int64 
 4   hp         1025 non-null   int64 
 5   attack     1025 non-null   int64 
 6   defense    1025 non-null   int64 
 7   s_attack   1025 non-null   int64 
 8   s_defense  1025 non-null   int64 
 9   speed      1025 non-null   int64 
 10  type       1025 non-null   object
 11  evo_set    1025 non-null   int64 
 12  info       1025 non-null   object
dtypes: int64(10), object(3)
memory usage: 104.2+ KB


Selecting Features, normalizing, and scaling

In [28]:
features = ['hp', 'attack', 'defense', 's_attack', 's_defense', 'speed']

# Convert the features to numeric, coerce errors to NaN
df[features] = df[features].apply(pd.to_numeric, errors='coerce')

# Normalize the features
scaler = MinMaxScaler() # Just to normalize the data regardless of how normalized it looks to make sure none of the values dominate
X = scaler.fit_transform(df[features])

### Cosine Similarity 
Cosine similarity is used in this case because it measures angle rather than distance. This is useful for our analysis because we are looking for similarity in the features of the Pokemon, not the distance between them

In [51]:
similarity = cosine_similarity(X)
similarity_df = pd.DataFrame(similarity, index=df['name'], columns=df['name'])

similarity_df.head()

name,bulbasaur,ivysaur,venusaur,charmander,charmeleon,charizard,squirtle,wartortle,blastoise,caterpie,...,fezandipiti,ogerpon,archaludon,hydrapple,gouging-fire,raging-bolt,iron-boulder,iron-crown,terapagos,pecharunt
name,,,,,,,,,,,,,,,,,,,,,
bulbasaur,1.000000,0.999645,0.998910,0.972709,0.984395,0.990620,0.979569,0.985119,0.987879,0.793465,...,0.960562,0.928793,0.973056,0.970260,0.939872,0.979252,0.936701,0.992499,0.971881,0.947646
ivysaur,0.999645,1.000000,0.999745,0.975035,0.986269,0.991663,0.981880,0.987467,0.990351,0.803529,...,0.966666,0.934806,0.971162,0.966467,0.944636,0.977395,0.943227,0.993516,0.975485,0.950406
venusaur,0.998910,0.999745,1.000000,0.976766,0.987170,0.991810,0.984387,0.989805,0.992576,0.812692,...,0.971631,0.942066,0.970751,0.963428,0.950924,0.974055,0.949969,0.992510,0.978413,0.953660
charmander,0.972709,0.975035,0.976766,1.000000,0.997653,0.994183,0.951447,0.959640,0.962382,0.866638,...,0.951576,0.953676,0.962948,0.914381,0.937367,0.943008,0.962024,0.971417,0.940413,0.930419
charmeleon,0.984395,0.986269,0.987170,0.997653,1.000000,0.998989,0.961855,0.969584,0.972421,0.856439,...,0.956830,0.946423,0.969479,0.935017,0.938982,0.962156,0.956291,0.984549,0.954383,0.938643


In [30]:
# Top 5 pokemon with most similar features to Pikachu
similarity_df['pikachu'].sort_values(ascending=False)[1:6]


name
persian       0.994905
swellow       0.994652
elekid        0.993871
poliwag       0.992266
talonflame    0.991992
Name: pikachu, dtype: float64

In [31]:
# top 5 pokemon with least similar features to Pikachu
similarity_df['pikachu'].sort_values(ascending=True)[:5]

name
shuckle      0.356214
chansey      0.475970
happiny      0.502408
blissey      0.559130
pyukumuku    0.571549
Name: pikachu, dtype: float64

### Network Graphing
Now based on the similarity matrix, we can create a network graph. The nodes will represent the pokemon, and the edges will represent a passing threshold of similarity. The graph will be undirected, meaning that the edges do not have a direction. Nodes will be colored based on type, and the size will be based on number of connections.

In [32]:

# Create a graph
G = nx.Graph()

for idx, row in df.iterrows():
    G.add_node(row['name'], type=row['type'], evo_set=row['evo_set'])


In [33]:
top_k = 5
minimum_similarity = 0.95

for i in range(len(df)):
    sims = similarity[i]
    top_indices = np.argsort(sims)[-top_k-1:-1]  # Top 5 excluding self

    for j in top_indices:
        if sims[j] >= minimum_similarity:
            source = df.loc[i, 'name']
            target = df.loc[j, 'name']
            sim = sims[j]
            
            if not G.has_edge(source, target):
                G.add_edge(source, target, weight=sim)

In [47]:
# Pikachu's top 5 most similar pokemon
pikachu_similar = similarity_df['pikachu'].sort_values(ascending=False)[1:6]

print("Pikachu's top 5 most similar pokemon:")
for name, sim in pikachu_similar.items():
    print(f"{name}: {sim:.2f}")

Pikachu's top 5 most similar pokemon:
persian: 0.99
swellow: 0.99
elekid: 0.99
poliwag: 0.99
talonflame: 0.99


In [34]:
net = Network(notebook=True, height='800px', width='100%', bgcolor='#222222', font_color='white')

In [35]:
from networkx.algorithms.community import greedy_modularity_communities

communities = list(greedy_modularity_communities(G))
community_map = {name: i for i, comm in enumerate(communities) for name in comm}

In [36]:
# Degree centrality for node sizing
centrality = nx.degree_centrality(G)

# Add nodes to PyVis with centrality-based size
for node in G.nodes(data=True):
    name = node[0]
    poke_type = node[1]['type']
    size = centrality[name] * 200 + 10  # Scales size from 10–110

    net.add_node(name, 
                 label=name, 
                 title=f"Type: {poke_type}", 
                 group=community_map[name], 
                 size=size)

### Adding weights to be the edges (Similarity between nodes)

In [38]:
for source, target, data in G.edges(data=True): 
    net.add_edge(source, target, value=data['weight'], color='rgba(255,255,255,0.2)')

In [39]:
unique_types = df['type'].unique()
for i, t in enumerate(sorted(unique_types)):
    print(f'Group {i}: {t}')

Group 0: {bug,dark}
Group 1: {bug,electric}
Group 2: {bug,fairy}
Group 3: {bug,fighting}
Group 4: {bug,fire}
Group 5: {bug,flying}
Group 6: {bug,ghost}
Group 7: {bug,grass}
Group 8: {bug,ground}
Group 9: {bug,poison}
Group 10: {bug,psychic}
Group 11: {bug,rock}
Group 12: {bug,steel}
Group 13: {bug,water}
Group 14: {bug}
Group 15: {dark,dragon}
Group 16: {dark,fairy}
Group 17: {dark,fighting}
Group 18: {dark,fire}
Group 19: {dark,flying}
Group 20: {dark,ghost}
Group 21: {dark,grass}
Group 22: {dark,ground}
Group 23: {dark,ice}
Group 24: {dark,normal}
Group 25: {dark,poison}
Group 26: {dark,psychic}
Group 27: {dark,steel}
Group 28: {dark}
Group 29: {dragon,dark}
Group 30: {dragon,electric}
Group 31: {dragon,fighting}
Group 32: {dragon,fire}
Group 33: {dragon,flying}
Group 34: {dragon,ghost}
Group 35: {dragon,ground}
Group 36: {dragon,ice}
Group 37: {dragon,normal}
Group 38: {dragon,psychic}
Group 39: {dragon,water}
Group 40: {dragon}
Group 41: {electric,dark}
Group 42: {electric,dragon}


In [40]:
net.show('pokemon_similarity_graph.html')


pokemon_similarity_graph.html


# Graph Metrics

In [41]:
# Graph Size
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print(f"The network contains {num_nodes} Pokémon connected by {num_edges} high-similarity edges.")

The network contains 1025 Pokémon connected by 3562 high-similarity edges.


In [42]:
density = nx.density(G)
print(f'Density: {density:.4f}')

Density: 0.0068


In [43]:
num_components = nx.number_connected_components(G)
largest_component = max(nx.connected_components(G), key=len)
largest_component_size = len(largest_component)

print(f"There are {num_components} connected components in the network.")
print(f"The largest component contains {largest_component_size} Pokémon, showing most Pokémon form a single connected stat ecosystem.")

There are 5 connected components in the network.
The largest component contains 1019 Pokémon, showing most Pokémon form a single connected stat ecosystem.


In [44]:
degree_centrality = nx.degree_centrality(G)
top_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]

print("Top 5 Pokémon by degree centrality (most stat-similar connections):")
for name, score in top_degree:
    print(f"  {name} — connected to ~{int(score * (num_nodes - 1))} other Pokémon")

Top 5 Pokémon by degree centrality (most stat-similar connections):
  charizard — connected to ~14 other Pokémon
  hoppip — connected to ~14 other Pokémon
  rillaboom — connected to ~14 other Pokémon
  aggron — connected to ~13 other Pokémon
  patrat — connected to ~13 other Pokémon


In [45]:
betweenness = nx.betweenness_centrality(G)
top_betweenness = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)[:5]

print("Top 5 Pokémon by betweenness centrality (bridge Pokémon):")
for name, score in top_betweenness:
    print(f"  {name} — {score:.4f} (bridges many Pokémon together)")

Top 5 Pokémon by betweenness centrality (bridge Pokémon):
  eevee — 0.0362 (bridges many Pokémon together)
  seedot — 0.0337 (bridges many Pokémon together)
  spewpa — 0.0319 (bridges many Pokémon together)
  zarude — 0.0311 (bridges many Pokémon together)
  volcanion — 0.0307 (bridges many Pokémon together)


In [46]:
from networkx.algorithms.community import greedy_modularity_communities

communities = list(greedy_modularity_communities(G))
num_communities = len(communities)
largest_community_size = max(len(c) for c in communities)

print(f"Detected {num_communities} stat-based Pokémon communities.")
print(f"The largest community contains {largest_community_size} Pokémon — representing the most common stat archetype.")

Detected 13 stat-based Pokémon communities.
The largest community contains 216 Pokémon — representing the most common stat archetype.


In [70]:
# Ten instances of Pokemon in the smallest community
largest_community = max(communities, key=len)
print("Largest community Pokémon:")
#Frozenset to list
largest_community_list = list(largest_community)
for name in largest_community_list[-1:11:-1]:
    print(f"  {name} — {df[df['name'] == name]['type'].values[0]}")

Largest community Pokémon:
  pichu — {electric}
  kabutops — {rock,water}
  weedle — {bug,poison}
  rookidee — {flying}
  oinkologne — {normal}
  okidogi — {poison,fighting}
  banette — {ghost}
  primeape — {fighting}
  gouging-fire — {fire,dragon}
  arctibax — {dragon,ice}
  pinsir — {bug}
  sprigatito — {grass}
  furret — {normal}
  nidoran-m — {poison}
  patrat — {normal}
  krookodile — {ground,dark}
  slaking — {normal}
  unfezant — {normal,flying}
  dodrio — {normal,flying}
  bruxish — {water,psychic}
  inkay — {dark,psychic}
  urshifu — {fighting,dark}
  growlithe — {fire}
  ceruledge — {fire,ghost}
  pidove — {normal,flying}
  stoutland — {normal}
  solrock — {rock,psychic}
  zangoose — {normal}
  shinx — {electric}
  annihilape — {fighting,ghost}
  bisharp — {dark,steel}
  grubbin — {bug}
  bombirdier — {flying,dark}
  passimian — {fighting}
  corvisquire — {flying}
  sawk — {fighting}
  shiftry — {grass,dark}
  luxio — {electric}
  beedrill — {bug,poison}
  rufflet — {normal,f